In [5]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import time
import pickle
import copy
import logging
from collections import defaultdict
from multiprocessing import Pool
from IPython.display import Math
from sympy import Add, Expr, I, Mul, Pow, S, Symbol, diff, pi, preorder_traversal, simplify, latex
from sympy.physics.quantum import Commutator, IdentityOperator, HermitianOperator, TensorProduct, qapply
from sympy.physics.quantum.sho1d import SHOKet
from symqudit.transmon import Transmon
from symqudit.two_transmon_hamiltonian import (TwoTransmonHamiltonian, FrequencyDiff, sort_block_diagonal,
                                               to_dict, from_dict, dict_product)
from symqudit.schrieffer_wolff_expansion import SWExpansion, integrate_exp_term, integrate_expr
from symqudit.common import ketbra, get_expr_at_order, organize_by_denom, DiscreteBra, DiscreteKet, DiscreteOuterProduct

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [2]:
tr = Transmon('', 100, 0.1)

In [3]:
e0 = tr.eigenvalue(0, 3)
for level in range(1, 4):
    display(Math(fr'\frac{{E_{level} - E_0}}{{\omega_h}} = ' + latex(((tr.eigenvalue(level, 3) - e0) / tr._omegah).expand())))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [27]:
def organize_by_ket(state):
    ket_coeffs = defaultdict(lambda: S.Zero)
    for term in state.expand().args:
        c, nc = term.args_cnc()
        ket_coeffs[nc[0].args] += Mul(*c)

    states = []
    for ket in sorted(ket_coeffs.keys()):
        coeff = ket_coeffs[ket]
        states.append(latex(coeff.expand() * SHOKet(*ket)))
        
    return '+'.join(states)

for level in range(4):
    display(Math(organize_by_ket(tr.eigenstate(level, 2))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
for row, col in [(0, 1), (1, 2), (2, 3), (0, 3)]:
    display(Math(fr'\mu_{{{row}{col}}} = ' + latex(tr.phase_matrix_element(row, col, 2))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [34]:
for row, col in [(0, 1), (1, 2), (2, 3), (0, 3)]:
    display(Math(fr'\nu_{{{row}{col}}} = ' + latex((I * tr.charge_matrix_element(row, col, 2)).expand())))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>